In [ ]:
# Boilerplate that all notebooks reuse:
from analysis_common import *

%matplotlib inline

# Kernel analysis

In [ ]:
df = read_ods("./results.ods", "matsum-kernel")

#############################################
# Disregard the store result for the kernel #
df.drop(index=df[df["RESULT_IN_NVRAM"] == 1].index, columns="RESULT_IN_NVRAM", inplace=True)
df.rename(columns={"INPUT_IN_NVRAM": "EXEC_IN_NVRAM"}, inplace=True)
#############################################

expand_modes(df)

order = ['DRAM', 'AD', 'MM (hot)', 'MM (cold)']

sns.barplot(x='MODE', y='TIMING',
            data=df[(df.BLOCKSIZE == 1000)],
            capsize=0.1,
            order=order,
            palette=custom_kernel_palette(5))
plt.title("Submatrix size: 1000x1000 (small object)")
plt.ylabel("kernel exec time (s)")
plt.xticks(rotation=25, horizontalalignment='right')
plt.show()

sns.barplot(x='MODE', y='TIMING',
            data=df[(df.BLOCKSIZE == 7000)],
            capsize=0.1,
            order=order,
            palette=custom_kernel_palette(5))
plt.title("Submatrix size: 7000x7000 (big object)")
plt.ylabel("kernel exec time (s)")
plt.xticks(rotation=25, horizontalalignment='right')
plt.show()

In [ ]:
df.loc[(df.BLOCKSIZE == 1000), "NORMALIZED"] = df.TIMING 
df.loc[(df.BLOCKSIZE == 7000), "NORMALIZED"] = df.TIMING / (7*7)

ax = sns.barplot(x='MODE', y='NORMALIZED',
                 data=df,
                 capsize=0.1,
                 order=order,
                 hue="BLOCKSIZE",
                 palette="Set2")

kernel_plot_tweaks(ax, 7*7, legend_title="Submatrix blocksize", rotate=True)

plt.show()


In [ ]:
kernel_times = df.groupby(["BLOCKSIZE", "MODE"]).min()
kernel_times

# Matsumresults analysis

In [ ]:
df = read_ods("./results.ods", "matsum-app")
expand_modes(df)

In [ ]:
for bs in [1000, 7000]:
    df.loc[(df.BLOCKSIZE == bs) & (df.MODE == "DRAM"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(bs, "DRAM"), "TIMING"]
    df.loc[(df.BLOCKSIZE == bs) & (df.MODE == "AD (volatile result)"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(bs, "AD"), "TIMING"]
    df.loc[(df.BLOCKSIZE == bs) & (df.MODE == "AD (store result)"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(bs, "AD"), "TIMING"]
    df.loc[(df.BLOCKSIZE == bs) & (df.MODE == "DAOS (volatile result)"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(bs, "DRAM"), "TIMING"]
    df.loc[(df.BLOCKSIZE == bs) & (df.MODE == "DAOS (store result)"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(bs, "DRAM"), "TIMING"]

df.loc[(df.BLOCKSIZE == 1000) 
       & (df.MATRIX_SIDE == 42) 
       & (df.MODE == "MM"),
       "ATOM_KERNEL"] = kernel_times.loc[(1000, "MM (hot)"), "TIMING"]
df.loc[(df.BLOCKSIZE == 7000) 
       & (df.MATRIX_SIDE == 6)
       & (df.MODE == "MM"),
       "ATOM_KERNEL"] = kernel_times.loc[(7000, "MM (hot)"), "TIMING"]
df.loc[(df.BLOCKSIZE == 1000) 
       & (df.MATRIX_SIDE == 84) 
       & (df.MODE == "MM"),
       "ATOM_KERNEL"] = kernel_times.loc[(1000, "MM (cold)"), "TIMING"]
df.loc[(df.BLOCKSIZE == 7000) 
       & (df.MATRIX_SIDE == 12)
       & (df.MODE == "MM"),
       "ATOM_KERNEL"] = kernel_times.loc[(7000, "MM (cold)"), "TIMING"]

df["KERNEL_TIME"] = df["MATRIX_SIDE"]**2 * df["ATOM_KERNEL"]

# Sanity check
null_values = df[df.isnull().values]
if len(null_values) > 0:
    print('There are null values, check null_values variable')

In [ ]:
null_values

In [ ]:
small = (
    ((df.BLOCKSIZE == 1000) & (df.MATRIX_SIDE == 42)) |
    ((df.BLOCKSIZE == 7000) & (df.MATRIX_SIDE == 6))
)

big = (
    ((df.BLOCKSIZE == 1000) & (df.MATRIX_SIDE == 84)) |
    ((df.BLOCKSIZE == 7000) & (df.MATRIX_SIDE == 12))
)

order = ["DRAM", "AD (volatile result)", "AD (store result)", "DAOS (volatile result)", "DAOS (store result)"]

ax = sns.barplot(x='MODE', y="TIMING",
                 data=df[small],
                 capsize=0.1,
                 order=order,
                 palette="Set1",
                 hue=df.BLOCKSIZE)

bottom = sns.barplot(x='MODE', y="KERNEL_TIME",
            data=df[small],
            capsize=0,
            order=order,
            palette="Set2",
            hue=df.BLOCKSIZE)

xlabel_tweaks(ax, 3, 2, rotate=True)
legend_tweaks(bottom, ["small objects", "big objects", "kernel comp."], placement='upper left')
plt.title("Small dataset")
plt.show()

ax = sns.barplot(x='MODE', y="TIMING",
                 data=df[big],
                 capsize=0.1,
                 order=order,
                 palette="Set1",
                 hue=df.BLOCKSIZE)

annotate_dram(ax)

bottom = sns.barplot(x='MODE', y="KERNEL_TIME",
            data=df[big],
            capsize=0,
            order=order,
            palette="Set2",
            hue=df.BLOCKSIZE)

xlabel_tweaks(ax, 3, 2, rotate=True)
legend_tweaks(bottom, ["small objects", "big objects", "kernel comp."], placement='upper left')
plt.title("Big dataset")
plt.show()
